딥러닝 코드를 작성하다 보면, 신경망 모델 자체를 코딩하는 시간보다 그 모델을 훈련시키는 코드를 짜는데 더 오랜 시간이 걸리기 마련입니다. 데이터 입력을 준비하는 부분도 이에 해당합니다.

$토치텍스트^{torchText}$는 자연어 처리 문제 또는 텍스트에 관한 머신러닝이나 딥러닝을 수행하는 데이터를 읽고 전처리하는 코드를 모아둔 라이브러리입니다. 텍스트 분류나 언어 모델, 그리고 기계 번역의 경우에도 토치텍스트를 활용하여 쉽게 텍스트 파일을 읽어내 훈련에 사용합니다.

### 예제 코드

자연어 처리 분야에서 주로 쓰이는 학습 데이터는 크게 3가지 형태로 분류할 수 있습니다.

<br></br>
![](./images/4-9-2-inputoutput.jpg)
<br></br>

따라서 우리는 이 3가지 종류의 데이터 형태를 다루는 방법을 실습합니다. 토치텍스트는 훨씬 복잡하고 정교한 함수들을 제공합니다. 여기서는 제공되는 함수들의 사용을 최소화하여 복잡하지 않고 간단한 방식으로 구현해보고자 합니다.

보통 *Field*라는 클래스를 통해 우리가 읽고자 하는 텍스트 파일 내의 필드를 먼저 정의합니다. 텍스트 파일 내에서 탭을 사용하여 필드를 구분하는 방식을 자연어 처리 분야의 입력에서 가장 많이 사용합니다. 쉼표의 경우에는 텍스트 내부에 많이 포함될 수 있으므로, 쉼표를 구분 문자로 사용하는 것은 위험한 선택이 될 가능성이 높습니다. 이렇게 정의된 각 필드를 Dataset 클래스를 통해 읽어들입니다.

읽어들인 코퍼스는 미리 주어진 미니배치 크기에 따라서 나뉠 수 있도록 이터레이터에 들어갑니다. 미니배치를 구성하는 과정에서 미니배치 내에서 문장의 길이가 다를 경우에는 필요에 따라 문장의 앞 또는 뒤에 $패딩^{padding}(PAD)$을 삽입합니다. 이 패딩은 추후 소개할 BOS, EOS와 함께 하나의 단어 또는 토큰과 같은 취급을 받습니다. 이후에 훈련 코퍼스에 대해 어휘 사전을 만들어 각 토큰을 숫자로 맵핑하는 작업을 수행하면 됩니다. 

#### 코퍼스와 레이블 읽기

첫번째 예제는 한줄에서 클래스와 텍스트가 탭으로 구분된 데이터의 입력을 받는 내용입니다. 이런 예제는 주로 **텍스트 분류**에서 사용됩니다.

In [1]:
## https://stackoverflow.com/a/66517960
from torchtext.legacy import data

class DataLoader(object):
    
    def __init__(self, train_fn, valid_fn,
                 batch_size = 64,
                 device = 1,
                 max_vocab = 999999,
                 min_freq = 1,
                 use_eos = False,
                 shuffle = True):
        
        super(DataLoader, self).__init__()
        
        ## Define field of the input file
        ## The input file consists of two fields.
        self.label = data.Field(sequential = False,
                                use_vocab = True,
                                unk_token = None)
        self.text = data.Field(use_vocab = True,
                               batch_first = True,
                               include_lengths = False,
                               eos_token = "<EOS>" if use_eos else None)
        
        ## Those defined two columns will be delimited by TAB
        ## Thus, we use TabularDataset to load two columns in the input file.
        ## We would have two separate input files: train_fn, valid_fn
        ## Files consist of two columns: label field and text field.
        
        train, valid = data.TabularDataset.splits(path = "",
                                                  train = train_fn,
                                                  validation = valid_fn,
                                                  format = "tsv",
                                                  fields = [('label', self.label),
                                                            ('text', self.text)])
        
        ## Those loaded dataset would be feeded into each iterator:
        ## train iterator and valid iterator
        ## We sort input sentences by length, to group similar lengths
        self.train_iter, self.valid_iter = data.BucketIterator.splits((train, valid),
                                                                      batch_size = batch_size,
                                                                      device = "cuda:%d" % device if device >=0 else "cpu",
                                                                      shuffle = shuffle,
                                                                      sort_key = lambda x: len(x.text),
                                                                      sort_within_batch = True)
        
        ## At last, we make a vocabulary for label and text field.
        ## It is making mapping table between words and indice.
        self.label.build_vocab(train)
        self.text.build_vocab(train, max_size = max_vocab, min_freq = min_freq)

#### 코퍼스 읽기

이 예제는 한 라인이 텍스트로만 채워져 있을때를 위한 코드입니다. 주로 **언어 모델**을 훈련시키는 상황에서 쓸 수 있습니다. **LanguageModelDataset**을 통해 미리 정의된 필드를 텍스트 파일에서 읽어들입니다. 이때 각 문장의 길이에 따라 정렬을 통해 비슷한 길이의 문장끼리 미니배치를 만들어줍니다. 이 작업을 통해서 매우 상이한 길이의 문장들이 하나의 미니배치에 묶여 훈련 시간에서 손해보는 것을 방지합니다.

In [2]:
from torchtext.legacy import data, datasets

PAD, BOS, EOS = 1, 2, 3

class DataLoader():
    
    def __init__(self,
                 train_fn,
                 valid_fn,
                 batch_size = 64,
                 device = "cpu",
                 max_vocab = 99999999,
                 max_length = 255,
                 fix_length = None,
                 use_bos = True,
                 use_eos = True,
                 shuffle = True):
        
        super(DataLoader, self).__init__()
        
        self.text = data.Field(sequential = True,
                               use_vocab = True,
                               batch_first = True,
                               include_lengths = True,
                               fix_length = fix_length,
                               init_token = "<BOS>" if use_bos else None,
                               eos_token = "<EOS>" if use_eos else None)
        
        train = LanguageModelDataset(path = train_fn,
                                     fields = [("text", self.text)],
                                     max_length = max_length)
        
        valid = LanugageModelDataset(path = valid_fn,
                                     fields = [("text", self.text)],
                                     max_length = max_length)
        
        self.train_iter = data.BucketIterator(train,
                                              batch_size = batch_size,
                                              device = "cude%d" % device if device >= 0 else "cpu",
                                              shuffle = shuffle,
                                              sort_key = lambda x: -len(x.text),
                                              sort_within_batch = True)
        
        self.valid_iter = data.BucketIterator(valid,
                                              batch_size = batch_size,
                                              device = "cude%d" % device if device >= 0 else "cpu",
                                              shuffle = False,
                                              sort_key = lambda x: -len(x.text),
                                              sort_within_batch = True)       
        
        self.text.build_vocab(train, max_size = max_vocab)

In [3]:
class LanugageModelDataset(data.Dataset):
    
    def __init__(self, path, fields, max_length = None, **kwargs):
        if not isinstance(fields[0], (tuple, list)):
            fields = [("text", fields[0])]
            
        examples = []
        with open(path) as f:
            for line in f:
                line = line.strip()
                
                if max_length and max_length < len(line.split()):
                    continue
                
                if line != "":
                    examples.append(data.Example.fromlist([line], fields))
                    
        super(LanugageModelDataset, self).__init__(examples, fields, **kwargs)

#### 병렬 코퍼스 읽기

다음 예제는 **텍스트로만 채워진 2개의 파일을 동시에 입력 데이터**로 읽어들이는 코드입니다. 이때 두 파일의 코퍼스는 **병렬 데이터**로 취급되어 같은 라인끼리 맵핑되어야 하므로, 같은 라인 수로 채워져 있어야 합니다.

주로 기계번역이나 요약등에 사용될 수 있습니다. 탭을 사용하여 하나의 파일에서 2개의 *$열^{column}$에 각 언어의 문장을 표현하는 것도 한가지 방법이 될 수 있습니다. 그렇다면 앞서 소개한 `TabularDataset` 클래스를 이용하면 됩니다. 또한 `LanguageModelDataset`과 마찬가지로 길이에 따라서 미니배치를 구성합니다.

In [4]:
import os
from torchtext.legacy import data, datasets

PAD, BOS, EOS = 1, 2, 3

class DataLoader():
    
    def __init__(self,
                 train_fn = None,
                 valid_fn = None,
                 exts = None,
                 batch_size = 64,
                 device = "cpu",
                 max_vocab = 99999999,
                 max_length = 255,
                 fix_length = None,
                 use_bos = True,
                 use_eos = True,
                 shuffle = True,
                 dsl = False):
        
        super(DataLoader, self).__init__()
        
        self.src = data.Field(sequential = True,
                              use_vocab = True,
                              batch_first = True,
                              include_lengths = True,
                              fix_length = fix_length,
                              init_token = "<BOS>" if dsl else None,
                              eos_token = "<EOS>" if dsl else None)
        
        self.tgt = data.Field(sequential = True,
                              use_vocab = True,
                              batch_first = True,
                              include_lengths = True,
                              fix_length = fix_length,
                              init_token = "<BOS>" if dsl else None,
                              eos_token = "<EOS>" if dsl else None)
        
        if train_fn is not None and valid_fn is not None and exts is not None:
            train = TranslationDataset(path = train_fn,
                                       exts = exts,
                                       fields = [("src", self.src),
                                                 ("tgt", self.tgt)],
                                       max_length = max_length)
            
            valid = TranslationDataset(path = valid_fn,
                                       exts = exts,
                                       fields = [("src", self.src),
                                                 ("tgt", self.tgt)],
                                       max_length = max_length)

            self.train_iter = data.BucketIterator(train,
                                                  batch_size = batch_size,
                                                  device = "cude%d" % device if device >= 0 else "cpu",
                                                  shuffle = shuffle,
                                                  sort_key = lambda x: len(x.tgt) + (max_length * len(x.src)),
                                                  sort_within_batch = True)
            
            self.valid_iter = data.BucketIterator(valid,
                                                  batch_size = batch_size,
                                                  device = "cude%d" % device if device >= 0 else "cpu",
                                                  shuffle = False,
                                                  sort_key = lambda x: len(x.tgt) + (max_length * len(x.src)),
                                                  sort_within_batch = True)
            
            self.src.build_vocab(train, max_size = max_vocab)
            self.tgt.build_vocab(train, max_size = max_vocab)
            
        def load_vocab(self, src_vocab, tgt_vocab):
            self.src.vocab = src_vocab
            self.tgt.vocab = tgt_vocab                                                  

In [5]:
class TranslationDataset(data.Dataset):
    
    @staticmethod
    def sort_key(ex):
        return data.interleave_keys(len(ex.src), len(ex.trg))
    
    def __init__(self, path, exts, fields, max_length = None, **kwargs):
        if not isinstance(fields[0], (tuple, list)):
            fields = [("src", fields[0]), ("trg", fields[1])]
            
        if not path.endswith("."):
            path += "."
            
        src_path, trg_path = tuple(os.path.expanduser(path + x) for x in exts)
        
        examples = []
        with open(src_path, encoding = "utf-8") as src_file, open(trg_path, encoding = "utf-8") as trg_file:
            for src_line, trg_line in zip(src_file, trg_file):
                src_line, trg_line = src_line.strip(), trg_line.strip()
                
                if max_length and max_length < max(len(src_line.split()),
                                                   len(trg_line.split())):
                    continue
                    
                if src_line != "" and trg_line != "":
                    examples.append(data.Example.fromlist([src_line, trg_line], fields))
        
        super().__init__(examples, fields, **kwargs)